# Libraries

In [1]:
import spacy
import pandas as pd
import os
from spacy.tokens import DocBin
from tqdm import tqdm
import ast

# Read train file

In [101]:
filename = open(r'D:\R & D PROJECT\Train_text\train_data.txt', 'r')
train_data=filename.read()
train = ast.literal_eval(train_data)

In [104]:
train[1]

('villa housing scheme construction reserve single 2 storey house bathroom tvhall living room model house ready inspection cctv security gym also available 3 block land available ate 0773209667 Malinda 0719663300 priyantha o 712298486 parakandeniya hettikanda road kokiskade junction kadawatha frank rude ,',
 {'entities': [(0, 5, 'Type'),
   (181, 191, 'Contact'),
   (192, 199, 'Advertiser'),
   (200, 210, 'Contact'),
   (223, 232, 'Contact'),
   (211, 220, 'Advertiser'),
   (233, 246, 'Location'),
   (263, 272, 'Location')]})

# Model building

In [106]:
#nlp = spacy.load('en_core_web_sm')
nlp = spacy.blank('en') # new spacy model

db = DocBin()

for text, annot in tqdm(train): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)

os.chdir(r'C:\Users\uer\Documents\Python Scripts\nlp_model_property')
db.to_disk('./train.spacy') 

  9%|▉         | 26/297 [00:00<00:02, 91.58it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity


 92%|█████████▏| 274/297 [00:00<00:00, 255.20it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity


100%|██████████| 297/297 [00:00<00:00, 436.76it/s]


# filling config

In [ ]:
# python -m spacy init fill-config base_config.cfg config.cfg

# Run training

In [ ]:
# python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./train.spacy

# Import model

In [2]:
nlp_model = spacy.load(r'C:\Users\uer\Documents\Python Scripts\nlp_model_property\output\model-best')

# test data

In [3]:
filename = open(r'D:\R & D PROJECT\Test_text\test_data.txt', 'r')
test_data = filename.read()
test_txt = [data.replace('\n','') for data in test_data.split(',')]
test_txt[:5]

['2 storey house negonbo 2 storey air conditioned house thushara Hiawatha 2 mind negombo Colombo road 5 mind negombo town lobby living dinning ki kitchen pantry balcony 4 bedroom 3 bathroom 3000 sq foot solar power hot water e system nmatap water upper tank garbage service 3 time week land area 12 perch price 28000000 negotiable tel 0765331254 ',
 'ratmalana house ratmalana borupana road 118 perch land house 3 bedroom 2 toilet 1 hall dining Hal pantry kitchen wall right round front garden rear garden 2 court yard walking distance car gills super kudu r 3000000 negotiable broker genuine buyer 0773890545 ',
 'katubedda pbalwis per era Hiawatha 300 Galle road 4 spacious bedroom 2 bathroom large living dining area spacious pantry outside extra kitchen toilet garage parapet wall around 19 perch asking price 360 00000 inspection appointment ovary wry aye ',
 'luxury house Colombo 6 2500 sqft 4 bedroom ac 3 bathroom hot water parking 2 rolled shutter gate oz unfurnished upstairs house 225 k p

In [219]:
prop_all = []
for test in test_txt:
    doc = nlp_model(test)
    property_lst = []
    for ent in doc.ents:
        tup=ent.label_, ent.text
        property_lst.append(tup)
    prop_all.append(list(set(property_lst)))

In [244]:
print(prop_all[1])

[('Location', 'ratmalana'), ('Bathroom', '2 toilet'), ('Bedroom', '3 bedroom'), ('LandSize', '118 perch'), ('Advertiser', 'rear'), ('Type', 'house'), ('Contact', '0773890545'), ('Price', 'r 3000000')]


In [220]:
new_prop_lst = []
for tup_l in prop_all:
    new_tup_lst = []
    for tup in tup_l:
        if new_tup_lst:
            letters = []
            for element in new_tup_lst:
                letters.append(element[0])
            if tup[0] not in letters:
                new_tup_lst.append(tup)
        else:
            new_tup_lst.append(tup)
    new_prop_lst.append(new_tup_lst)

In [221]:
print(new_prop_lst[1])

[('Location', 'ratmalana'), ('Bathroom', '2 toilet'), ('Bedroom', '3 bedroom'), ('LandSize', '118 perch'), ('Advertiser', 'rear'), ('Type', 'house'), ('Contact', '0773890545'), ('Price', 'r 3000000')]


# Find Property type and Append to list

In [222]:
ptype_lst = []
for typ_lst in new_prop_lst:
    found = False
    for ptyp in typ_lst:
        if ptyp[0] == 'Type':
            ptype_lst.append(ptyp[1])
            found = True
    if not found:
        ptype_lst.append('NaN')
len(ptype_lst)

158

# Find Location

In [224]:
ploc_lst = []
for loc_lst in new_prop_lst:
    found = False
    for ploc in loc_lst:
        if ploc[0] == 'Location':
            ploc_lst.append(ploc[1])
            found = True
    if not found:
        ploc_lst.append('NaN')
print(ploc_lst[1])

ratmalana


# Find Land size

In [225]:
lsize_lst = []
for size_lst in new_prop_lst:
    found = False
    for s in size_lst:
        if s[0] == 'LandSize':
            lsize_lst.append(s[1])
            found = True
    if not found:
        lsize_lst.append('NaN')
len(lsize_lst)

158

# Find property size

In [226]:
psize_lst = []
for psz_lst in new_prop_lst:
    found = False
    for psize in psz_lst:
        if psize[0] == 'PropertySize':
            psize_lst.append(psize[1])
            found = True
    if not found:
        psize_lst.append('NaN')
len(psize_lst)

158

# Find Bedrooms

In [227]:
bedroom_lst = []
for br_lst in new_prop_lst:
    found = False
    for bed in br_lst:
        if bed[0] == 'Bedroom':
            bedroom_lst.append(bed[1])
            found = True
    if not found:
        bedroom_lst.append('NaN')
len(bedroom_lst)

158

# Find Bathrooms

In [228]:
bathroom_lst = []
for bthrm_lst in new_prop_lst:
    found = False
    for bath in bthrm_lst:
        if bath[0] == 'Bathroom':
            bathroom_lst.append(bath[1])
            found = True
    if not found:
        bathroom_lst.append('NaN')
len(bathroom_lst)

158

# Find Advertiser

In [231]:
advertiser_lst = []
for advtr_lst in new_prop_lst:
    found = False
    for advt in advtr_lst:
        if advt[0] == 'Advertiser':
            advertiser_lst.append(advt[1])
            found = True
    if not found:
        advertiser_lst.append('NaN')
print(advertiser_lst[0])

thushara


# Find Contact Number

In [232]:
contact_lst = []
for cont_lst in new_prop_lst:
    found = False
    for cont in cont_lst:
        if cont[0] == 'Contact':
            contact_lst.append(cont[1])
            found = True
    if not found:
        contact_lst.append('NaN')
len(contact_lst)

158

# Find Price

In [239]:
pprice_lst = []
for price_lst in new_prop_lst:
    found = False
    for price in price_lst:
        if price[0] == 'Price':
            pprice_lst.append(price[1])
            found = True
    if not found:
        pprice_lst.append('NaN')
len(pprice_lst)

158

# Create DataFrame

In [234]:
data_dic = {'Property_type':ptype_lst,
                    'Address':ploc_lst,
                    'Area_of_Land':lsize_lst,
                    'Bedrooms':bedroom_lst,
                    'Bathrooms':bathroom_lst,
                    'Property_size':psize_lst,
                    'Advertiser':advertiser_lst,
                    'contact_num':contact_lst,
                    'Price':pprice_lst}

In [237]:
data = pd.DataFrame.from_dict(data_dic,orient='index')
data=data.transpose()
data.head(5)

,Property_type,Address,Area_of_Land,Bedrooms,Bathrooms,Property_size,Advertiser,contact_num,Price
0,house,Colombo,12 perch,4 bedroom,3 bathroom,3000 sq,thushara,NaN,price 28000000
1,house,ratmalana,118 perch,3 bedroom,2 toilet,NaN,rear,0773890545,r 3000000
2,NaN,katubedda,19 perch,NaN,2 bathroom,NaN,NaN,NaN,360 00000
3,house,Colombo 6,NaN,4 bedroom,3 bathroom,2500 sqft,NaN,0773867704,NaN
4,NaN,piliyandala,6 p,2,bathroom,NaN,NaN,NaN,NaN


In [241]:
extract_data = data.to_csv(r'D:\R & D PROJECT\Test_text\extract_data.csv',index=False)

In [249]:
data = pd.read_csv(r'D:\R & D PROJECT\Test_text\extract_data.csv')
data.head()

,Property_type,Address,Area_of_Land,Bedrooms,Bathrooms,Property_size,Advertiser,contact_num,Price
0,house,Colombo,12 perch,4 bedroom,3 bathroom,3000 sq,thushara,NaN,price 28000000
1,house,ratmalana,118 perch,3 bedroom,2 toilet,NaN,rear,773890545.0,r 3000000
2,NaN,katubedda,19 perch,NaN,2 bathroom,NaN,NaN,NaN,360 00000
3,house,Colombo 6,NaN,4 bedroom,3 bathroom,2500 sqft,NaN,773867704.0,NaN
4,NaN,piliyandala,6 p,2,bathroom,NaN,NaN,NaN,NaN
